## AutoGen with Google Cloud Storage Integration
### Overview
This Jupyter notebook demonstrates how to integrate AutoGen (multi-agent conversation framework) with Google Cloud Storage (GCS) for automated file management tasks.

## What is AutoGen?
AutoGen is Microsoft's cutting-edge framework for creating multi-agent conversational AI systems. It enables multiple AI agents to collaborate, debate, and solve complex problems through structured conversations.
Key AutoGen Features:

* Multi-Agent Conversations: Multiple AI agents can talk to each other
* Role Specialization: Each agent can have specific roles and capabilities
* Function Calling: Agents can execute real-world functions and tools
* Conversation Flow Control: Sophisticated conversation management
* Human-in-the-Loop: Optional human oversight and intervention

## What is Google Cloud Storage (GCS)?
Google Cloud Storage is Google's object storage service for storing and accessing data on Google Cloud Platform. It's designed for storing any amount of data and retrieving it as often as needed.
Key GCS Features:

* Scalable Storage: Store from gigabytes to petabytes
* Global Accessibility: Access data from anywhere in the world
* Security: Enterprise-grade security and access controls
* Cost-Effective: Multiple storage classes for different use cases
* Integration: Works seamlessly with other Google Cloud services

Dependencies and Installation

In [ ]:
!pip install autogen pyautogen google-cloud-storage google-generativeai python-dotenv langchain_google_genai ag2[gemini] vertexai

## Required Imports

In [45]:
import os
import google.generativeai as genai
import autogen
from autogen import ConversableAgent
from google.cloud import storage
from pathlib import Path

## Configuration Setup
1. Google Cloud Authentication

In [46]:
print("Setting up GCP credentials...")
api_key = "Your JSON file name"  # Your JSON file name
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = api_key

Setting up GCP credentials...


In [47]:
print("Setting up Gemini API...")
GEMINI_API_KEY = "Your GEMINI API key"  # Your API key
os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY
genai.configure(api_key=GEMINI_API_KEY)

Setting up Gemini API...


In [48]:
print("Testing connections...")
try:
    # Test GCP
    client = storage.Client(project="Gen-ai")
    print(f"✅ GCP connected: {client.project}")

    # Test Gemini
    model = genai.GenerativeModel('gemini-1.5-flash')
    test = model.generate_content("Hello")
    print(f"✅ Gemini connected: {test.text[:30]}...")
except Exception as e:
    print(f"❌ Connection error: {e}")
    print("Please check your API keys and JSON file")

Testing connections...
✅ GCP connected: Gen-ai
✅ Gemini connected: Hello there! How can I help yo...


In [49]:
print("Configuring AutoGen for Gemini...")
config_list = [
    {
        "model": "gemini-1.5-flash",
        "api_key": GEMINI_API_KEY,
        "api_type": "google"
    }
]

Configuring AutoGen for Gemini...


### STEP 2: SET BASE DIRECTORY

In [50]:
BASE_DIR = Path("/content")  # For Google Colab
# For local use: BASE_DIR = Path(r"C:\your\path\here")
print(f"Base directory: {BASE_DIR}")

Base directory: /content


### STEP 3: DEFINE GCP FUNCTIONS

In [51]:
def check_bucket_permissions():
    """Check bucket permissions"""
    try:
        client = storage.Client(project="Gen-ai")
        bucket = client.get_bucket("bucket_demo7")
        permissions_to_test = [
            'storage.objects.create',
            'storage.objects.get',
            'storage.objects.list',
            'storage.buckets.get'
        ]
        allowed_permissions = bucket.test_iam_permissions(permissions_to_test)
        return f"Bucket permissions: {allowed_permissions}"
    except Exception as e:
        return f"Error checking permissions: {str(e)}"

In [52]:
def list_gcs_buckets():
    """List GCS buckets"""
    try:
        client = storage.Client(project="Gen-ai")
        buckets = list(client.list_buckets())
        bucket_names = [bucket.name for bucket in buckets]
        return f"Found {len(bucket_names)} buckets: {', '.join(bucket_names)}"
    except Exception as e:
        try:
            client = storage.Client(project="Gen-ai")
            bucket = client.get_bucket("bucket_demo7")
            return f"Cannot list all buckets, but can access: {bucket.name}"
        except Exception as bucket_error:
            return f"Error: {str(e)}"

In [53]:
def upload_to_gcs_debug(file_name):
    """Upload file to GCS with debugging"""
    try:
        # Handle filename variations
        possible_names = [
            file_name,
            file_name.replace('_', ' '),
            file_name.replace(' ', '_'),
            file_name.replace('eer2', 'eer_2'),
            file_name.replace('eer_2', 'eer2')
        ]

        file_path = None
        actual_filename = None

        # Find the file
        for name in possible_names:
            test_path = BASE_DIR / name
            if test_path.exists():
                file_path = test_path
                actual_filename = name
                break

        if file_path is None:
            if BASE_DIR.exists():
                pdf_files = [f.name for f in BASE_DIR.iterdir() if f.suffix.lower() == '.pdf']
                return f"❌ File '{file_name}' not found. Available: {pdf_files}"
            else:
                return f"❌ Directory '{BASE_DIR}' does not exist"

        print(f"Debug: Found file: {file_path}")
        print(f"Debug: Filename: {actual_filename}")
        print(f"Debug: Size: {file_path.stat().st_size} bytes")

        # Upload to GCS
        client = storage.Client(project="Gen-ai")
        bucket = client.get_bucket("bucket_demo7")
        blob = bucket.blob(file_name)
        blob.upload_from_filename(str(file_path))

        return f"✅ Successfully uploaded '{actual_filename}' as '{file_name}' to bucket_demo7"
    except Exception as e:
        return f"❌ Upload error: {str(e)}"

print("GCP functions defined successfully!")

GCP functions defined successfully!


### STEP 4: CREATE AUTOGEN AGENTS

In [54]:
print("Creating AutoGen agents...")

# 7a. Create GCP Agent (uses Gemini 1.5 Flash)
gcp_agent = ConversableAgent(
    name="GCPAgent",
    system_message="""You are a GCP Storage assistant. You can:
1. check_bucket_permissions() - Check permissions
2. list_gcs_buckets() - List buckets
3. upload_to_gcs_debug(file_name) - Upload files

Use these functions to help users with GCS tasks. Always call the actual functions, don't write code blocks.""",
    llm_config={
        "config_list": config_list,
        "functions": [
            {
                "name": "check_bucket_permissions",
                "description": "Check bucket permissions",
                "parameters": {"type": "object", "properties": {}, "required": []}
            },
            {
                "name": "list_gcs_buckets",
                "description": "List buckets",
                "parameters": {"type": "object", "properties": {}, "required": []}
            },
            {
                "name": "upload_to_gcs_debug",
                "description": "Upload a file to GCS",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "file_name": {"type": "string", "description": "File to upload"}
                    },
                    "required": ["file_name"]
                }
            }
        ]
    },
    human_input_mode="NEVER"
)

Creating AutoGen agents...


### STEP 5: Create User Agent (executes functions)

In [56]:
user_agent = ConversableAgent(
    name="User",
    system_message="You execute functions and return results.",
    llm_config=False,
    human_input_mode="NEVER",
    function_map={
        "check_bucket_permissions": check_bucket_permissions,
        "list_gcs_buckets": list_gcs_buckets,
        "upload_to_gcs_debug": upload_to_gcs_debug
    }
)
print("✅ Agents created successfully!")

✅ Agents created successfully!


### STEP 6: TEST THE SYSTEM

In [57]:
print("\n" + "="*50)
print("TESTING GEMINI + AUTOGEN + GCP")
print("="*50)

# 8a. Test functions directly first
print("\n1. DIRECT FUNCTION TESTS:")
print("-" * 30)
print("Testing permissions:", check_bucket_permissions())
print("Testing buckets:", list_gcs_buckets())
print("Testing upload:", upload_to_gcs_debug("temp_eer_2.pdf"))

# 8b. Test with AutoGen conversation
print("\n2. AUTOGEN CONVERSATION TEST:")
print("-" * 35)
try:
    result = user_agent.initiate_chat(
        gcp_agent,
        message="Please check bucket permissions and then upload temp_eer_2.pdf file.",
        max_turns=2
    )
    print("\n✅ Conversation completed successfully!")
except Exception as e:
    print(f"\n❌ Conversation error: {str(e)}")
    print("Note: If direct function calls work above, your GCP setup is correct.")


TESTING GEMINI + AUTOGEN + GCP

1. DIRECT FUNCTION TESTS:
------------------------------
Testing permissions: Bucket permissions: ['storage.buckets.get', 'storage.objects.create', 'storage.objects.get', 'storage.objects.list']
Testing buckets: Cannot list all buckets, but can access: bucket_demo7
Debug: Found file: /content/temp_eer_2.pdf
Debug: Filename: temp_eer_2.pdf
Debug: Size: 1229101 bytes
Testing upload: ✅ Successfully uploaded 'temp_eer_2.pdf' as 'temp_eer_2.pdf' to bucket_demo7

2. AUTOGEN CONVERSATION TEST:
-----------------------------------
User (to GCPAgent):

Please check bucket permissions and then upload temp_eer_2.pdf file.

--------------------------------------------------------------------------------


GCPAgent (to User):

Okay, I'll first check the bucket permissions.

```tool_code
check_bucket_permissions()
```

Assuming the permissions check is successful, I'll proceed with uploading the file.

```tool_code
upload_to_gcs_debug("temp_eer_2.pdf")
```


--------------------------------------------------------------------------------
User (to GCPAgent):



--------------------------------------------------------------------------------


GCPAgent (to User):

I need a task to perform.  I can check bucket permissions (`check_bucket_permissions()`), list your GCS buckets (`list_gcs_buckets()`), or upload a file (`upload_to_gcs_debug(file_name)`).  Please tell me what you'd like me to do.


--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (b37fc0d3-0f8d-47b7-b2bb-60d5ffb884c3): Maximum turns (2) reached

✅ Conversation completed successfully!
